I am helping to analyze some initial MBDSeq data from a Dungeness crab OA experiment, run by NOAA (Krista Nichols' group, with Mac) on the MiSeq to do a quick QC before sending the libraries off for full sequencing.  

#### Summary of MBDBS library prep 
Tissue used is genomic DNA (gDNA) from Dungeness crab gill tissue. 
Libraries were prepared by Sam White of the Roberts Lab using the  Pico Methyl-Seq Library Prep Kit (ZymoResearch). See the following lab notebook entries from Sam White for some details on the library prepTissue shearing, day 1/2
 - [Tissue shearing, day 2/2](https://robertslab.github.io/sams-notebook/2020/10/27/DNA-Shearing-M.magister-gDNA-Additional-Shearing-CH05-01_21-CH07-11-and-Bioanalyzer.html)  
 - [MBD Selection, day 1/3](https://robertslab.github.io/sams-notebook/2020/10/28/MBD-Selection-M.magister-Sheared-Gill-gDNA-8-of-24-Samples-Set-1-of-3.html)  
 - [MBD Selection, day 2/3](https://robertslab.github.io/sams-notebook/2020/11/02/MBD-Selection-M.magister-Sheared-Gill-gDNA-8-of-24-Samples-Set-2-of-3.html)  
 - [MBD Selection, day 3/3](https://robertslab.github.io/sams-notebook/2020/11/03/MBD-Selection-M.magister-Sheared-Gill-gDNA-16-of-24-Samples-Set-3-of-3.html)  
 - [BSseq library prep](https://robertslab.github.io/sams-notebook/2020/11/24/MBD-BSseq-Library-Prep-M.magister-MBD-selected-DNA-Using-Pico-Methyl-Seq-Kit.html)  
 - [QC with Bioanalyzer](https://robertslab.github.io/sams-notebook/2020/11/25/Bioanalyzer-M.magister-MBD-BSseq-Libraries.html)  
 - [Library quantification with Qubit](https://robertslab.github.io/sams-notebook/2020/12/02/Library-Quantification-M.magister-MBD-BSseq-Libraries-with-Qubit.html)  


#### My task: 
Run the MiSeq data through a trimming/mapping and alignment pipeline so we can get an idea of mapping efficiency and bisulfite conversion rate (i.e. CHG methylation from Bismark). The pipeline is based on the one developed by the MethCompare group ([check out their repo](https://github.com/hputnam/Meth_Compare)), and which I tested for the WGBS data. See my WGBS pipeline in this Jupyter Notebook [Notebook-01_Exploring-WGBS-data.ipynb](https://nbviewer.jupyter.org/github/laurahspencer/C.magister_methyl-oa/blob/master/notebooks/Notebook-01_Exploring-WGBS-data.ipynb). 

Steps include: 

  - Use TrimGalore to trim and filter reads:
    - Trim Illumina adapters  
    - Hard trim 10bp from 5' and 3' ends  
    - Filter for quality using default Phred score: 20  
    - Specify that we have paired-end files    
  - Run FastQC and generate MultiQC report  
  - Use Bismark to align reads to genome, deduplicate, call methylation state, and generate Bismark summary report  
    - minimum alignment score needed for an alignment to be considered "valid" (i.e. good enough to report) set to  `score_min L,0,-0.6`. See the [Bismark user guide](https://rawgit.com/FelixKrueger/Bismark/master/Docs/Bismark_User_Guide.html#ii-bismark-alignment-step) for details.  
    - Specify that the sequencing library was constructed in a non strand-specific manner, alignments to all four bisulfite strands will be reported (using `--non_directional`). This is the suggested setting for Pico-Methyl kit.   

The pipeline was executed on Mox. It's been a couple years since working on Mox, so I first brushed up on its general use via the helpful [Mox GitHub repo](https://github.com/RobertsLab/hyak_mox/wiki). 

#### Initial Hiccup: cannot get Jupyter Notebook to work on Mox.  
  - I followed instructions on the [Roberts Mox Wiki](https://github.com/RobertsLab/hyak_mox/wiki/Jupyter-Notebooks) and on the [Mox website](https://wiki.cac.washington.edu/display/hyakusers/Mox_ipython_jupyter), but I kept getting "permission denied" after this step: 
  ```
  jupyter notebook --no-browser --port=8899
  OSError: [Errno 13] Permission denied: '/run/user/224081'
  ```
This step should have given me a "token", but it did not. I thought perhaps the Anaconda module listed on the Mox wiki Jupyter Notebook instructions was out of date. I looked at the anaconda versions installed in our shared programs/ directory, which was anaconda3, so perhaps I need to use a different module. To see available modules, I entered `module avail anaconda` and got this: 

```
------------------------------------- /sw/modules-1.775/modulefiles --------------------------------------
anaconda2_4.3.1  anaconda2_5.3  anaconda3_4.3.1  anaconda3_5.3  
```

I tried loading all four anaconda modules, but still could not get past the `jupyter notebook --no-browser --port=8899` step. So I could not move forward with using Jupyter Notebook. In an effort to make some headway anyway, I decided to NOT pursue Jupyter Notebook on Mox for this initial data processing. 

Instead, I will summarize my work here. 

### A summary of steps I took to run TrimGalore and Bismark jobs on Mox:   

#### Logged on to Mox 
Via `ssh lhs3@mox.hyak.uw.edu` then entered my UW pw and followed 2-step authentication.  

### Get Data

#### Transfer MiSeq data to Mox.   
  - Mac transferred the data to Gannet. First, I checked that I could access it. Yep! Logged on via browser using Gannet's IP address and my Gannet-specific password to view files.  
  - Checked the size of all the MiSeq data (3.5GB) on Gannet using the browser interface. My personal storage capacity on Mox is 10GB, our lab's storage is 5500GB, and the temporary storage is 200TB. Based on recommendations from the lab listed on the Mox Wiki, I will save all MiSeq-related files (raw data, filtered data, scripts) on the shared gscratch/srlab/lhs3 directory which does not get scrubbed.  
  - I initially tried transferring data from Gannet, but encountered a permissions error (see [this GitHub issue](https://github.com/RobertsLab/resources/issues/1052). Turns out the directory where the MiSeq data is saved is not public, so I cannot transfer from there. Instead ...      
  - The MiSeq data is also saved on Nightengales (on Owl), and Sam instructed me to tranfer it from there to Mox. So, I logged on to Owl via the terminal using: `ssh lhs3@128.95.149.83` (and my Owl-specific pw).  
  - I navigated to the nightengales directory, then copied all files in the C_magister subdirectory to Mox using rsync:     
```rsync --archive --progress --verbose --relative ./C_magister lhs3@mox.hyak.uw.edu:/gscratch/srlab/lhs3/inputs/```

#### Transferred Dungeness crab genome files to Mox   
To align MiSeq reads to the genome using Bismark, I will need the genome file!  Also, I need the index that Mac previously built via Bismark. I used/saved these files on Ostrich in the Spring when I processed the WGBS data. So, it was simplest for me to log on to Ostrich and use rsync to transfer files to Mox. 

Transferred genome fasta file: 
```
(base) ostrich:~ lhs3$ rsync --archive --progress --verbose /Users/lhs3/Documents/C.magister_methyl0oa/data/pilon_scaffolds.fasta lhs3@mox.hyak.uw.edu:/gscratch/srlab/lhs3/inputs/C_magister/
```

Transferred genome index files: 
```
(base) ostrich:~ lhs3$ rsync --archive --progress --verbose /Users/lhs3/Documents/C.magister_methyl0oa/data/Bisulfite_Genome/ lhs3@mox.hyak.uw.edu:/gscratch/srlab/lhs3/inputs/C_magister/Bisulfite_Genome/ 
```

#### Write slurm scripts to run Trim/Filter and Alignment jobs on Mox 

I wrote Mox-specific scripts by perusing those Shelly Trigg developed, which are listed on her [lab notebook entries](https://shellytrigg.github.io/315th-post/), and referring to the script associated with the [MethCompare pipeline](https://github.com/hputnam/Geoduck_Meth/blob/master/code/Geoduck_Meth_Pipeline.md) (both very helpful!). Here are my final slurm scripts that I used: 

  - Trim/Filter using TrimGalore: [20201213_Cmag_trim.sh](https://raw.githubusercontent.com/laurahspencer/C.magister_methyl-oa/master/scripts/20201213_Cmag_trim.sh)  
  - Align, deduplicate, and call methylation status using Bismark: [20201214_Cmag_bismark-align.sh](https://raw.githubusercontent.com/laurahspencer/C.magister_methyl-oa/master/scripts/20201214_Cmag_bismark-align.sh)

NOTE: there are some path issues with the Bismark script. I had to set the working directory as the one containing my trimmed/filtered read files (which is different than what the MethCompare folks do). **I need to figure out how exactly I can run Bismark from a directory other than that containing the read files.** 

I transferred my scripts from my personal computer over to Mox using `rsync`.  

#### Get  packages  

Many packages are already installed on Mox in `/gscratch/srlab/programs/` for shared use. However, when I first tried to use TrimGalore on Mox, it threw an error because it could not find the dependency `Cutadapt`. I saw that Cutadapt was listed on the Mox wiki as an installed software, but I didn't see it in the Mox /gscratch/srlab/programs/ directory, or in the anaconda3/bin subdirectory, or in my $PATH. Turns out `Cutadapt` is a tricky wicket - see [this github issue](https://github.com/RobertsLab/resources/issues/1053) where Sam explains some of the install issues (I also remembering this being an annoying install when I used it on Ostrich). 

I was finally able to run TrimGalore after installing several software packages: 
 
 1) Installed the latest version of Miniconda for Lenox. Downloaded the package "Python 3.8 | Miniconda3 Linux 64-bit" from [here](https://docs.conda.io/en/latest/miniconda.html#installing) using `wget` into my personal programs directory (/gscratch/srlab/lhs3/programs/). Installed Miniconda using directions listed [here](https://conda.io/projects/conda/en/latest/user-guide/install/linux.html).  
    - NOTE: I _should_ have tried using the Miniconda already installed in `/gscratch/srlab/programs/`. See the last comment in [this GitHub issue](https://github.com/RobertsLab/resources/issues/1053) for details_.  
 
 2) Set up new channels using instructions listed [here](https://bioconda.github.io/user/install.html)
```
conda config --add channels defaults
conda config --add channels bioconda
conda config --add channels conda-forge
```
 
 3) Installed `cutadapt` using the conda install directions listed [here](https://cutadapt.readthedocs.io/en/stable/installation.html)  
 
 4) While the cutadaptenv was activated, identified the path to the `cutadapt` program on Mox using `whereis cutadapt` so I could specify location in my job script (although not sure if that was necessary).   
 
 5) Added the most current version of FastQC on Mox to my $PATH using 
`export PATH=$PATH:/gscratch/srlab/programs/fastqc_v0.11.9`. This was definitely necessary, since TrimGalore does not have an option to specify FastQC's path.   


#### Run the TrimGalore script to trim and filter MiSeq data, and generate MultiQC report 

To run the trim/filter job, I navigated to `/gscratch/srlab/lhs3/jobs/` on Mox (where I transferred my slurm scripts), and executed the following code: `sbatch -p srlab -A srlab 20201213_Cmag_trim.sh`  

The job (#588844) took ~ 21 minutes; [here's the slurm output](https://raw.githubusercontent.com/laurahspencer/C.magister_methyl-oa/master/qc-processing/MiSeq-QC/trimmed/slurm-588844.out). Oddly not all the samples ran correctly. Here's an example of the encountered error that occurred when trying to run sample CH10-11_S20_L001_R1_001:  

```
Using an excessive number of cores has a diminishing return! It is recommended not to exceed 8 cores per trimming process (you asked for 8 cores). Please consider re-specifying
Path to Cutadapt set as: '/gscratch/home/lhs3/miniconda3/envs/cutadaptenv/bin/cutadapt' (user defined)
Cutadapt seems to be working fine (tested command '/gscratch/home/lhs3/miniconda3/envs/cutadaptenv/bin/cutadapt --version')
Cutadapt version: 3.1
Could not detect version of Python used by Cutadapt from the first line of Cutadapt (but found this: >>>#!/bin/sh<<<)
Letting the (modified) Cutadapt deal with the Python version instead
Proceeding with 'gzip' for compression. PLEASE NOTE: Using multi-cores for trimming with 'gzip' only has only very limited effect! (see here: https://github.com/FelixKrueger/TrimGalore/issues/16#issuecomment-458557103)
To increase performance, please install 'pigz' and run again

No quality encoding type selected. Assuming that the data provided uses Sanger encoded Phred scores (default)

Output will be written into the directory: /gscratch/srlab/lhs3/outputs/20201213/trimmed/
Writing report to '/gscratch/srlab/lhs3/outputs/20201213/trimmed/CH10-11_S20_L001_R1_001.fastq.gz_trimming_report.txt'

SUMMARISING RUN PARAMETERS
==========================
Input filename: /gscratch/srlab/lhs3/inputs/C_magister/CH10-11_S20_L001_R1_001.fastq.gz
Trimming mode: paired-end
Trim Galore version: 0.6.6
Cutadapt version: 3.1
Python version: could not detect
Number of cores used for trimming: 8
Quality Phred score cutoff: 20
Quality encoding type selected: ASCII+33
Adapter sequence: 'AGATCGGAAGAGC' (Illumina TruSeq, Sanger iPCR; user defined)
Maximum trimming error rate: 0.1 (default)
Minimum required adapter overlap (stringency): 1 bp
Minimum required sequence length for both reads before a sequence pair gets removed: 20 bp
All Read 1 sequences will be trimmed by 10 bp from their 5' end to avoid poor qualities or biases
All Read 2 sequences will be trimmed by 10 bp from their 5' end to avoid poor qualities or biases (e.g. M-bias for BS-Seq applications)
All Read 1 sequences will be trimmed by 10 bp from their 3' end to avoid poor qualities or biases
All Read 2 sequences will be trimmed by 10 bp from their 3' end to avoid poor qualities or biases
Running FastQC on the data once trimming has completed
Running FastQC with the following extra arguments: '--outdir FastQC --threads 28'
Output file(s) will be GZIP compressed

Cutadapt seems to be fairly up-to-date (version 3.1). Setting -j 8
Writing final adapter and quality trimmed output to CH10-11_S20_L001_R1_001_trimmed.fq.gz


  >>> Now performing quality (cutoff '-q 20') and adapter trimming in a single pass for the adapter sequence: 'AGATCGGAAGAGC' from file /gscratch/srlab/lhs3/inputs/C_magister/CH10-11_S20_L001_R1_001.fastq.gz <<< 
This is cutadapt 3.1 with Python 3.8.6
Command line parameters: -j 8 -e 0.1 -q 20 -O 1 -a AGATCGGAAGAGC /gscratch/srlab/lhs3/inputs/C_magister/CH10-11_S20_L001_R1_001.fastq.gz
Processing reads on 8 cores in single-end mode ...
ERROR: Traceback (most recent call last):
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/cutadapt/pipeline.py", line 575, in run
    self.send_to_worker(chunk_index, chunk)
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 135, in __exit__
    self.close()
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 321, in close
    self._raise_if_error(allow_sigterm=allow_sigterm)
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 344, in _raise_if_error
    raise OSError("{} (exit code {})".format(message, retcode))
OSError: gzip: /gscratch/srlab/lhs3/inputs/C_magister/CH10-11_S20_L001_R1_001.fastq.gz: invalid compressed data--format violated (exit code 1)

ERROR: Traceback (most recent call last):
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/cutadapt/pipeline.py", line 575, in run
    self.send_to_worker(chunk_index, chunk)
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 135, in __exit__
    self.close()
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 321, in close
    self._raise_if_error(allow_sigterm=allow_sigterm)
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 344, in _raise_if_error
    raise OSError("{} (exit code {})".format(message, retcode))
OSError: gzip: /gscratch/srlab/lhs3/inputs/C_magister/CH10-11_S20_L001_R1_001.fastq.gz: invalid compressed data--format violated (exit code 1)

ERROR: Traceback (most recent call last):
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/cutadapt/pipeline.py", line 575, in run
    self.send_to_worker(chunk_index, chunk)
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 135, in __exit__
    self.close()
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 321, in close
    self._raise_if_error(allow_sigterm=allow_sigterm)
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 344, in _raise_if_error
    raise OSError("{} (exit code {})".format(message, retcode))
OSError: gzip: /gscratch/srlab/lhs3/inputs/C_magister/CH10-11_S20_L001_R1_001.fastq.gz: invalid compressed data--format violated (exit code 1)

ERROR: Traceback (most recent call last):
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/cutadapt/pipeline.py", line 575, in run
    self.send_to_worker(chunk_index, chunk)
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 135, in __exit__
    self.close()
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 321, in close
    self._raise_if_error(allow_sigterm=allow_sigterm)
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 344, in _raise_if_error
    raise OSError("{} (exit code {})".format(message, retcode))
OSError: gzip: /gscratch/srlab/lhs3/inputs/C_magister/CH10-11_S20_L001_R1_001.fastq.gz: invalid compressed data--format violated (exit code 1)

ERROR: Traceback (most recent call last):
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/cutadapt/pipeline.py", line 575, in run
    self.send_to_worker(chunk_index, chunk)
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 135, in __exit__
    self.close()
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 321, in close
    self._raise_if_error(allow_sigterm=allow_sigterm)
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 344, in _raise_if_error
    raise OSError("{} (exit code {})".format(message, retcode))
OSError: gzip: /gscratch/srlab/lhs3/inputs/C_magister/CH10-11_S20_L001_R1_001.fastq.gz: invalid compressed data--format violated (exit code 1)

ERROR: Traceback (most recent call last):
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/cutadapt/pipeline.py", line 575, in run
    self.send_to_worker(chunk_index, chunk)
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 135, in __exit__
    self.close()
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 321, in close
    self._raise_if_error(allow_sigterm=allow_sigterm)
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/xopen/__init__.py", line 344, in _raise_if_error
    raise OSError("{} (exit code {})".format(message, retcode))
OSError: gzip: /gscratch/srlab/lhs3/inputs/C_magister/CH10-11_S20_L001_R1_001.fastq.gz: invalid compressed data--format violated (exit code 1)

ERROR: Traceback (most recent call last):
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/cutadapt/pipeline.py", line 646, in run
    (n, bp1, bp2) = self._pipeline.process_reads()
  File "/usr/lusers/lhs3/miniconda3/envs/cutadaptenv/lib/python3.8/site-packages/cutadapt/pipeline.py", line 331, in process_reads
    for read in self._reader:
  File "src/dnaio/_core.pyx", line 260, in fastq_iter
dnaio.exceptions.FastqFormatError: Error in FASTQ file at line 42036: Length of sequence and qualities differ

cutadapt: error: Error in FASTQ file at line 42036: Length of sequence and qualities differ
```
---

Luckily, before I went crazy trying to trouble shoot the issue, Sam also encountered an error when running the raw data through FastQC/MultiQC. Turns out some of the files were not transferred correctly - see [this GitHub issue](https://github.com/RobertsLab/resources/issues/1055) for more details. 

#### [Here is the MultiQC report for reads after trimming and filtering](https://nbviewer.jupyter.org/github/laurahspencer/C.magister_methyl-oa/blob/master/qc-processing/MiSeq-QC/trimmed/multiqc_report.html). Note that the samples that did not work in my pipeline are: 
  - S2 (CH01-14_S2)  
  - S4 (CH01-38_S4)  
  - S5 (CH03-04_S5)  
  - S6 (CH03-15_S6)  
  - S14 (CH07-24_S14)  
  - S20 (CH10-11_S20)  

Nevertheless, I continued the pipeline with the samples that did work.  

#### Run the Bismark script to align reads to genome, deduplicate, call methylation status, and generate Bismark summary report  

To run the alignment job, I navigated to `/gscratch/srlab/lhs3/jobs/` on Mox (where I transferred my slurm scripts), and executed the following code: `sbatch -p srlab -A srlab 20201214_Cmag_bismark-align.sh`  

The job (#595069) took 1.25hrs; [here's the slurm output](https://raw.githubusercontent.com/laurahspencer/C.magister_methyl-oa/master/qc-processing/MiSeq-QC/bismark/slurm-595069.out). 

Check out the [Bismark summary report (html version)](https://nbviewer.jupyter.org/github/laurahspencer/C.magister_methyl-oa/blob/master/qc-processing/MiSeq-QC/bismark/bismark_summary_report.html). And [here’s the .txt version of the Bismark report in .csv format](https://github.com/laurahspencer/C.magister_methyl-oa/blob/master/qc-processing/MiSeq-QC/bismark/bismark_summary_report.csv), where I calculated the \% methylation in various contexts (CpG, CHG, CHH). Here's a snapshot of the results: 

| File | Total Reads | Aligned Reads | Duplicate Reads (removed) | Unique Reads (remaining) | % Total reads mapped | % CpGs Methylated | % CHGs Methylated | % CHHs Methylated |
|-|-|-|-|-|-|-|-|-|
| CH01-06_S1 | 1,125,933 | 582,396 | 6,434 | 575,962 | 51% | 76% | 1.10% | 3.10% |
| CH01-22_S3 | 1,061,555 | 552,080 | 7,016 | 545,064 | 51% | 69% | 1.00% | 3.50% |
| CH03-33_S7 | 1,600,756 | 835,881 | 10,143 | 825,738 | 52% | 75% | 1.10% | 2.90% |
| CH05-01_S8 | 1,303,984 | 566,133 | 838 | 565,295 | 43% | 57% | 1.40% | 9.70% |
| CH05-06_S9 | 624,508 | 189,323 | 4,373 | 184,950 | 30% | 7% | 1.10% | 6.80% |
| CH05-21_S10 | 1,489,803 | 761,905 | 3,379 | 758,526 | 51% | 67% | 1.10% | 5.00% |
| CH05-24_S11 | 917,446 | 475,847 | 1,903 | 473,944 | 52% | 75% | 1.30% | 5.30% |
| CH07-06_S12 | 1,533,754 | 757,565 | 6,499 | 751,066 | 49% | 68% | 1.20% | 4.40% |
| CH07-11_S13 | 966,011 | 290,103 | 4,237 | 285,866 | 30% | 8% | 1.40% | 6.60% |
| CH09-02_S15 | 2,017,116 | 956,871 | 7,026 | 949,845 | 47% | 72% | 1.50% | 6.30% |
| CH09-13_S16 | 599,034 | 312,880 | 34,970 | 277,910 | 46% | 52% | 1.30% | 3.80% |
| CH09-28_S17 | 1,956,104 | 995,322 | 11,991 | 983,331 | 50% | 70% | 1.60% | 4.70% |
| CH10-01_S18 | 2,120,844 | 1,097,426 | 9,408 | 1,088,018 | 51% | 73% | 1.30% | 4.30% |
| CH10-08_S19 | 1,495,095 | 590,337 | 743 | 589,594 | 39% | 36% | 1.30% | 11.10% |
|  |  |  |  | MEAN | 45.9% | 57.5% | 1.3% | 5.5% |
|  |  |  |  | MEDIAN | 49.5% | 68.5% | 1.3% | 4.9% |

Summarized this work for Krista & Mac in a new Google Doc in the shared Google Drive. 


    
  

## MiSeq QC Processing, Take 2 (December 16th, 2020)

In the initial QC run only 14 of the samples made it through the entire pipeline (6 included some corrupt files). So, Today I re-ran the pipeline with all samples. 

- Re-transferred data from Owl/Nightengales to Mox   
- Combined my trim & align slurm scripts into one. It can be found [here](https://raw.githubusercontent.com/laurahspencer/C.magister_methyl-oa/master/scripts/20201216_Cmag_trim-align-dedup-callmeth.sh)  
- Made new Mox directory for this job: `/gscratch/srlab/lhs3/outputs/20201216/`    
- Transferred the new script to my `/gscratch/srlab/lhs3/jobs/` directory.  
- Ran the full trim/align job on Mox: **Job_id=617745 Name=Trim-Align-Cmag-MiSeq**  
  - TrimGalore and Bismark ran completely. I transferred all the report .txt and .html files to my computer/repo, then pushed to [GitHub](https://github.com/laurahspencer/C.magister_methyl-oa/tree/master/qc-processing/MiSeq-QC). See [slurm output](https://raw.githubusercontent.com/laurahspencer/C.magister_methyl-oa/master/qc-processing/MiSeq-QC/slurm-617745_full-MiSeq-run.out)  
  - However, FastQC/MultiQC didn't work on the full run. Figured out that I needed to re-add the FastQC program location to my \\$PATH (via `export PATH=$PATH:/gscratch/srlab/programs/fastqc_v0.11.9`). Maybe there's another way to permamently add that program to my PATH? Will look in to that. Anyway, I ran FastQC/MultQC on the trimmed reads in a separate job (Job_id=620042 Name=FastQC-Cmag-MiSeq, see [slurm output]()) using this [script](https://raw.githubusercontent.com/laurahspencer/C.magister_methyl-oa/master/scripts/20201217_Cmag_fastqc.sh)
  
### MultiQC report, all MiSeq data that has been trimmed/filtered: [multiqc_report.html](https://nbviewer.jupyter.org/github/laurahspencer/C.magister_methyl-oa/blob/master/qc-processing/MiSeq-QC/FastQC/multiqc_report.html)

### Bismark Summary Report, all MiSeq data: [HTML format](https://nbviewer.jupyter.org/github/laurahspencer/C.magister_methyl-oa/blob/master/qc-processing/MiSeq-QC/bismark/bismark_summary_report.html), and [csv format](https://github.com/laurahspencer/C.magister_methyl-oa/blob/master/qc-processing/MiSeq-QC/bismark/bismark_summary_report.csv).  

Here's a snapshot of the alignment / methylation results. Several samples have very low mCpG rates (<20%, compared to others with 60-70%). Those samples are: 

| File | % of Total Reads Unique | % CpGs Meth | % CHGs Meth | % CHHs Meth |
|-|-|-|-|-|
| CH01-06_S1 | 51.2% | 76.0% | 1.1% | 3.1% |
| CH01-14_S2 | 48.5% | 70.6% | 1.3% | 5.1% |
| CH01-22_S3 | 51.3% | 69.1% | 1.0% | 3.5% |
| CH01-38_S4 | 47.3% | 71.1% | 1.4% | 5.3% |
| -> CH03-04_S5 | 32.4% | 12.4% | 1.5% | 15.2% |
| -> CH03-15_S6 | 31.0% | 22.0% | 1.2% | 8.8% |
| CH03-33_S7 | 51.6% | 75.3% | 1.1% | 2.9% |
| CH05-01_S8 | 43.4% | 56.7% | 1.4% | 9.7% |
| -> CH05-06_S9 | 29.6% | 7.5% | 1.1% | 6.8% |
| CH05-21_S10 | 50.9% | 67.5% | 1.1% | 5.0% |
| CH05-24_S11 | 51.7% | 74.7% | 1.3% | 5.3% |
| CH07-06_S12 | 49.0% | 67.5% | 1.2% | 4.4% |
| -> CH07-11_S13 | 29.6% | 8.4% | 1.4% | 6.6% |
| -> CH07-24_S14 | 33.1% | 7.4% | 1.3% | 6.8% |
| CH09-02_S15 | 47.1% | 71.7% | 1.5% | 6.3% |
| CH09-13_S16 | 46.4% | 52.5% | 1.3% | 3.8% |
| CH09-28_S17 | 50.3% | 70.2% | 1.6% | 4.7% |
| CH10-01_S18 | 51.3% | 73.2% | 1.3% | 4.3% |
| ->  CH10-08_S19 | 39.4% | 36.0% | 1.3% | 11.1% |
| CH10-11_S20 | 49.0% | 70.8% | 1.3% | 4.9% |
|  |  |  |  |  |
|  | 44.2% | 53.0% | 1.3% | 6.2% |
|  | 47.9% | 68.3% | 1.3% | 5.2% |




